# 🛒 Retail Sales Insights (Databricks SQL + Dashboards)

**Goal**  
Turn raw e-commerce transactions into executive-grade insights: revenue trends, top products/customers, retention, churn, geography, and product affinities.

**Medallion Architecture**  
- **Bronze**: `production.bronze.retail_ecommerce_raw` (raw import)  
- **Silver**: `production.silver.retail_ecommerce_clean` (typed, cleaned, deduped)  
- **Gold**: Aggregated tables in `production.gold.*` for BI & dashboards

**Business Questions**  
1) Revenue trends (daily/monthly)  
2) Top products (revenue & units)  
3) Top customers & LTV  
4) Repeat customer rate  
5) AOV (average order value)  
6) Country revenue & customer base  
7) Churn (silent customers)  
8) Refunds/returns exposure (neg qty)  
9) Customer revenue tiers (80/20 view)  
10) Product affinity (bought-together pairs)

> Notes: This notebook creates gold tables and query views you can wire directly to Databricks SQL dashboards.


Sanity check environment

In [0]:
%sql
Use catalog production

In [0]:
%sql
select count(*) from production.bronze.retail_ecommerce_raw

In [0]:
%sql
select count(*) from production.silver.retail_ecommerce_clean

In [0]:
%sql
CREATE OR REPLACE VIEW production.silver.v_retail_clean_typed AS
SELECT
  CAST(InvoiceNo AS STRING)                        AS invoice_no,
  CAST(StockCode AS STRING)                        AS stock_code,
  TRIM(Description)                                AS description,
  CAST(Quantity AS INT)                            AS quantity,
  -- Try explicit format first, then fallback
  COALESCE(TRY_TO_TIMESTAMP(InvoiceDate,'M/d/yyyy H:mm'),
           TRY_TO_TIMESTAMP(InvoiceDate))          AS invoice_ts,
  CAST(UnitPrice AS DECIMAL(10,2))                 AS unit_price,
  CAST(CustomerID AS STRING)                       AS customer_id,
  TRIM(Country)                                    AS country,
  CAST(Quantity AS DECIMAL(18,2)) * 
  CAST(UnitPrice AS DECIMAL(18,2))                 AS revenue,
  DATE(COALESCE(TRY_TO_TIMESTAMP(InvoiceDate,'M/d/yyyy H:mm'),
                TRY_TO_TIMESTAMP(InvoiceDate)))    AS invoice_date,
  YEAR(COALESCE(TRY_TO_TIMESTAMP(InvoiceDate,'M/d/yyyy H:mm'),
                TRY_TO_TIMESTAMP(InvoiceDate)))    AS invoice_year,
  MONTH(COALESCE(TRY_TO_TIMESTAMP(InvoiceDate,'M/d/yyyy H:mm'),
                 TRY_TO_TIMESTAMP(InvoiceDate)))   AS invoice_month
FROM production.silver.retail_ecommerce_clean;


GOLD: Daily Sales


In [0]:
%sql
create or replace table production.gold.retail_daily_sales as 
select 
  invoice_date                  AS sales_date,
  COUNT(DISTINCT invoice_no)    AS orders,
  SUM(quantity)                 AS items_sold,
  SUM(revenue)                  AS revenue
FROM production.silver.v_retail_clean_typed
GROUP BY invoice_date
ORDER BY sales_date;

GOLD 2: Monthly Sales

In [0]:
%sql
-- MAKE_DATE(invoice_year, invoice_month, 1) constructs the first day of each month (e.g., 2024-06-01) from the year and month columns.
CREATE OR REPLACE TABLE production.gold.retail_monthly_sales AS
SELECT
  MAKE_DATE(invoice_year, invoice_month, 1) AS month_start,
  COUNT(DISTINCT invoice_no)                AS orders,
  SUM(quantity)                             AS items_sold,
  SUM(revenue)                              AS revenue
FROM production.silver.v_retail_clean_typed
GROUP BY MAKE_DATE(invoice_year, invoice_month, 1)
ORDER BY month_start;

GOLD 3: Customer LTV

In [0]:
%sql
CREATE OR REPLACE TABLE production.gold.retail_customer_ltv AS
SELECT
  customer_id,
  COUNT(DISTINCT invoice_no)                 AS orders,
  SUM(revenue)                               AS total_spent,
  -- First purchase date for the customer
  MIN(invoice_date)                          AS first_order_date,
  -- Most recent purchase date for the customer
  MAX(invoice_date)                          AS last_order_date
FROM production.silver.v_retail_clean_typed
GROUP BY customer_id;

GOLD 4: Product Sales

In [0]:
%sql
CREATE OR REPLACE TABLE production.gold.retail_product_sales AS
SELECT
  stock_code,
  COALESCE(NULLIF(TRIM(description),''), 'UNKNOWN')  AS product_name,
  SUM(quantity)                                      AS total_qty,
  SUM(revenue)                                       AS total_revenue,
  COUNT(DISTINCT customer_id)                        AS unique_customers
FROM production.silver.v_retail_clean_typed
GROUP BY stock_code, COALESCE(NULLIF(TRIM(description),''), 'UNKNOWN')
ORDER BY total_revenue DESC;


GOLD 5: Country Sales

In [0]:
%sql
CREATE OR REPLACE TABLE production.gold.retail_country_sales AS
SELECT
  country,
  SUM(revenue)                    AS total_revenue,
  COUNT(DISTINCT customer_id)     AS customers,
  COUNT(DISTINCT invoice_no)      AS orders
FROM production.silver.v_retail_clean_typed
GROUP BY country
ORDER BY total_revenue DESC;


Business Questions

Q1. Revenue trend (daily)

In [0]:
%sql
SELECT sales_date, revenue, orders, items_sold
FROM production.gold.retail_daily_sales
ORDER BY sales_date;


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Observation: Revenue fluctuates daily with visible spikes around November–December. Some dates show zero or negative sales, often tied to returns.
Insight: Sales are seasonal, with holiday peaks driving a disproportionate share of revenue.
Action: Plan inventory, staffing, and promotions to align with holiday peaks. Investigate zero-revenue days for operational gaps.

Q2. Revenue trend (monthly)


In [0]:
%sql
SELECT month_start, revenue, orders, items_sold
FROM production.gold.retail_monthly_sales
ORDER BY month_start;


Databricks visualization. Run in Databricks to view.

Observation: Clear monthly upward trend in late-year months, then a decline post-holidays.

Insight: Business is seasonal; Q4 contributes the bulk of sales.

Action: Front-load advertising and logistics planning into Q4. Smooth revenue in other months via promotions and bundles.

Top products by revenue

In [0]:
%sql
SELECT stock_code, product_name, total_revenue, total_qty, unique_customers
FROM production.gold.retail_product_sales
ORDER BY total_revenue DESC
LIMIT 15;


Databricks visualization. Run in Databricks to view.

Observation: A handful of SKUs (e.g., “Paper Craft Sets,” “Party Banners,” “Ceramic Mug”) account for >20% of total revenue.

Insight: The catalog follows the 80/20 rule (Pareto principle).

Action: Prioritize supply chain and marketing for top-selling SKUs. Negotiate bulk pricing with suppliers.

Top customers & LTV

In [0]:
%sql
SELECT customer_id, orders, total_spent, first_order_date, last_order_date
FROM production.gold.retail_customer_ltv
ORDER BY total_spent DESC
LIMIT 15;


Databricks visualization. Run in Databricks to view.

Observation: The top 10 customers spend 10–15x more than the average. Some accounts have hundreds of orders.

Insight: Revenue is concentrated in a small but powerful customer base.

Action: Launch loyalty/VIP programs to retain high-value customers. Assign account managers for top 1%.

Repeat-customer rate (KPI)

In [0]:
%sql
SELECT
  ROUND(100.0 * SUM(CASE WHEN orders > 1 THEN 1 ELSE 0 END) / COUNT(*), 2) AS repeat_customer_pct
FROM production.gold.retail_customer_ltv;


Observation: Roughly 65.58% of customers placed more than one order.
Insight: Decent retention, but majority are one-time buyers.
Action: Implement post-purchase reactivation campaigns (emails, coupons, product recommendations).

Average Order Value (AOV)

In [0]:
%sql
-- AOV overall and by month
WITH cust_orders AS (
  SELECT invoice_no, SUM(revenue) AS order_value,
         DATE_TRUNC('month', MIN(invoice_date)) AS month_bucket
  FROM production.silver.v_retail_clean_typed
  GROUP BY invoice_no
)
SELECT 
  ROUND(AVG(order_value),2) AS aov_overall
FROM cust_orders;


WITH cust_orders AS (
  SELECT invoice_no, SUM(revenue) AS order_value,
         DATE_TRUNC('month', MIN(invoice_date)) AS month_bucket
  FROM production.silver.v_retail_clean_typed
  GROUP BY invoice_no
)
SELECT month_bucket, ROUND(AVG(order_value),2) AS aov_monthly
FROM cust_orders
GROUP BY month_bucket
ORDER BY month_bucket;


Databricks visualization. Run in Databricks to view.

Observation: Average basket size is around £400–£650 (varies monthly).
Insight: Customers are buying in bulk (possibly B2B wholesalers).
Action: Incentivize larger orders by offering free shipping or discounts slightly above current AOV threshold.

Country revenue & customers

In [0]:
%sql
SELECT country, total_revenue, customers, orders
FROM production.gold.retail_country_sales
ORDER BY total_revenue DESC
LIMIT 15;


Databricks visualization. Run in Databricks to view.

Observation: United Kingdom dominates revenue (>80%), with smaller contributions from Netherlands, Germany, France, etc.

Insight: The business is UK-centric with limited international penetration.

Action: Expand into EU markets with localized marketing. Investigate shipping/logistics partnerships.

Churn (silent customers)

In [0]:
%sql
-- Define "silent" as no purchases in the last N days of dataset (e.g., 90)
WITH bounds AS (
  SELECT MAX(invoice_date) AS max_d FROM production.silver.v_retail_clean_typed
),
flag AS (
  SELECT l.*,
         CASE WHEN DATEDIFF((SELECT max_d FROM bounds), last_order_date) >= 90 THEN 1 ELSE 0 END AS is_silent_90d
  FROM production.gold.retail_customer_ltv l
)
SELECT 
  SUM(is_silent_90d) AS silent_customers_90d,
  COUNT(*)           AS total_customers,
  ROUND(100.0 * SUM(is_silent_90d) / COUNT(*), 2) AS silent_pct_90d
FROM flag;


Observation: About 33% of customers went silent after their first order, with no repeat activity in 90 days.
Insight: Churn is high, hurting long-term revenue.
Action: Launch win-back campaigns for silent customers (discount codes, targeted offers).

Refunds/returns impact (use Bronze to catch negatives)

In [0]:
%sql
WITH typed AS (
  SELECT
    CAST(InvoiceNo AS STRING)         AS invoice_no,
    CAST(Quantity AS INT)             AS quantity,
    COALESCE(TRY_TO_TIMESTAMP(InvoiceDate,'M/d/yyyy H:mm'),
             TRY_TO_TIMESTAMP(InvoiceDate)) AS invoice_ts,
    CAST(UnitPrice AS DECIMAL(10,2))  AS unit_price
  FROM production.bronze.retail_ecommerce_raw
),
calc AS (
  SELECT
    invoice_no,
    quantity,
    unit_price,
    quantity * unit_price AS line_amount
  FROM typed
)
SELECT
  SUM(CASE WHEN quantity < 0 THEN line_amount ELSE 0 END)        AS refunds_amount,     -- negative
  SUM(CASE WHEN quantity >= 0 THEN line_amount ELSE 0 END)       AS sales_amount,
  ROUND(100.0 * ABS(SUM(CASE WHEN quantity < 0 THEN line_amount ELSE 0 END)) 
        / NULLIF(SUM(CASE WHEN quantity >= 0 THEN line_amount ELSE 0 END),0), 2)        AS refunds_pct_of_sales
FROM calc;


Databricks visualization. Run in Databricks to view.

Observation: Returns/refunds make up about 9% of gross sales (via negative quantities).

Insight: Return rates are significant, cutting into profitability.

Action: Audit top-returned SKUs; improve product descriptions, packaging, and quality control.

Customer revenue tiers (80/20 lens)

In [0]:
%sql
WITH ranked AS (
  SELECT
    customer_id,
    total_spent,
    NTILE(10) OVER (ORDER BY total_spent DESC) AS decile -- D1 = top 10%
  FROM production.gold.retail_customer_ltv
)
SELECT 
  decile,
  COUNT(*)                               AS customers,
  ROUND(SUM(total_spent),2)              AS revenue,
  ROUND(100.0 * SUM(total_spent) / SUM(SUM(total_spent)) OVER (), 2) AS pct_of_total
FROM ranked
GROUP BY decile
ORDER BY decile;


Observation: Top 10% of customers contribute 50–60% of revenue. Bottom 50% of customers contribute <10%.

Insight: Classic Pareto distribution — business depends heavily on elite customers.

Action: Prioritize retention of top deciles, while experimenting with cost-effective ways to grow mid/low-value customers.

Product affinity (bought-together)

In [0]:
%sql
-- Pairs of products on the same invoice; filter out self-pairs and rare pairs
WITH base AS (
  SELECT DISTINCT invoice_no, stock_code
  FROM production.silver.v_retail_clean_typed
),
pairs AS (
  SELECT a.stock_code AS sku_a, b.stock_code AS sku_b, COUNT(*) AS together
  FROM base a
  JOIN base b
    ON a.invoice_no = b.invoice_no
   AND a.stock_code < b.stock_code
  GROUP BY a.stock_code, b.stock_code
)
SELECT *
FROM pairs
WHERE together >= 20            -- adjust threshold
ORDER BY together DESC
LIMIT 25;


Databricks visualization. Run in Databricks to view.

Observation: Products like “Party Banner” + “Gift Wrap” often appear in the same basket.
Insight: Cross-selling opportunities exist for frequently paired items.
Action: Bundle common pairs or recommend them at checkout to boost AOV.